In [27]:
#launched with
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
from fastcluster import linkage, pdist        #optimizing clustering tasks : pairwise & linkage
from Bio import SeqIO                         #sequence import outport
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import AlignIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.Cluster import kcluster
from Bio.Align.Applications import MuscleCommandline
from bioservices import *
from Bio import Phylo
from io import StringIO
from ete3 import Tree, TreeStyle
np.set_printoptions(precision=4,suppress=True)
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, fcluster, cophenet, set_link_color_palette
from scipy.spatial.distance import squareform
from fastcluster import linkage, pdist                  #optimizing clustering tasks : pairwise & linkage



In [28]:
# Create list of tuples (id, sequence string) from fasta file 

with open('apr11sequences.fasta') as fasta_file:  # Will close handle cleanly
    fastas = []
    for values in SimpleFastaParser(fasta_file):
        fastas.append(values)
        
# Form Pandas Data Frame from list

allSequences = pd.DataFrame(fastas, columns=['id','Sequence'])
print("All sequences shape: " + str(allSequences.shape))

# Import extended info table

infoGenbank = pd.read_csv("genBank sequence info.csv")
print("GenBank info shape: " + str(infoGenbank.shape))


All sequences shape: (470, 2)
GenBank info shape: (436, 8)


In [29]:
# Isolate the accession from allSequences.id column

allSequences["Accession"] = allSequences.id.str.slice(0,9)
allSequences["Accession"] = allSequences["Accession"]

#combine tables for sequences to & info bits
combinedTableTop = pd.merge(allSequences, infoGenbank, on="Accession")
print(combinedTableTop.shape)

(1, 10)


In [30]:
#create new accession column using first 8 bits of id column
allSequences["Accession"] = allSequences.id.str.slice(0,8)

#combine 
combinedTableBottom = pd.merge(allSequences, infoGenbank, on="Accession")
print(combinedTableBottom.shape)


(435, 10)


In [31]:
itemizedSequences = pd.concat([combinedTableTop, combinedTableBottom], ignore_index=True)
itemizedSequences.head

itemizedSequences = itemizedSequences.drop(columns = ['id', 'GenBank_Title', 'Isolation_Source', 'Release_Date', 'Host'])
itemizedSequences = itemizedSequences.reindex(columns=['Accession', 'Geo_Location', 'Collection_Date', 'Length', 'Sequence'])
itemizedSequences

,Accession,Geo_Location,Collection_Date,Length,Sequence
0,NC_045512,China,2019-12,29903,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
1,MT308704,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...
2,MT308703,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...
3,MT308702,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...
4,MT304479,USA: GA,3/3/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
...,...,...,...,...,...
431,MN988713,USA: Illinois,1/21/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
432,MN938384,China: Shenzhen,1/10/2020,29838,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...
433,MN975262,China,1/11/2020,29891,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
434,MN985325,USA,1/19/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...


In [32]:
aprAligns = []
handle = "aligned411.fasta"
aprAligned = AlignIO.read(handle, "fasta")
for alignment in aprAligned:
    aprAligns.append((alignment.id, str(alignment.seq)))


In [33]:
aprAlignments = pd.DataFrame(aprAligns, columns = ["Accession", "Alignment"])                        # creating dataframe from list 
aprAlignments

,Accession,Alignment
0,MT293180,----------------------------------------------...
1,MT322400,--------------------TTCCCAGGTAACAAACCAACCAACTT...
2,MT263387,----------------------------------------------...
3,MT263426,----------------------------------------------...
4,MT259284,----------------------------------------------...
...,...,...
465,MT163718,----TTGGTTGGTTTATACCTTCSCAGGTAACAAACCAACCAACTT...
466,MT322407,-----------------------------------------AACTT...
467,MT293185,----------------------------------------------...
468,MT259266,----------------------------------------------...


In [34]:
itemizedAligned = pd.merge(itemizedSequences, aprAlignments, on="Accession")
itemizedAligned.shape
itemizedAligned

,Accession,Geo_Location,Collection_Date,Length,Sequence,Alignment
0,NC_045512,China,2019-12,29903,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...
1,MT308704,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...
2,MT308703,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...
3,MT308702,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...
4,MT304479,USA: GA,3/3/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...
...,...,...,...,...,...,...
431,MN988713,USA: Illinois,1/21/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...
432,MN938384,China: Shenzhen,1/10/2020,29838,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...,------------------------------------CAACCAACTT...
433,MN975262,China,1/11/2020,29891,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...
434,MN985325,USA,1/19/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...


In [35]:
alignmentMatrix = np.asarray([np.fromstring(each, dtype=np.uint8) for each in itemizedAligned.Alignment])



/Users/lupine/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


In [36]:
alignLinks = linkage(alignmentMatrix, metric='euclidean', method='average')
print(alignLinks)

[[  0.     435.       0.       2.    ]
 [  5.      18.       0.       2.    ]
 [  6.      14.       0.       2.    ]
 ...
 [865.     866.     337.7836 423.    ]
 [265.     867.     407.8547  13.    ]
 [868.     869.     462.5851 436.    ]]


In [37]:
distances_euclidean = pdist(alignmentMatrix, metric='euclidean')
print(distances_euclidean)

[211.0995 210.751  207.0821 ...  68.9638  79.0253  96.2653]


In [38]:
avg_cophenet,coph_dists = cophenet(alignLinks,distances_euclidean)
print("Cophenetic correlation, euclidean distance with average linkage:",avg_cophenet)
#Great preservation of distance matrices!

Cophenetic correlation, euclidean distance with average linkage: 0.945009725128188


In [40]:
maxD = 310
dClusters = fcluster(alignLinks, max_d, criterion='distance')
itemizedAligned["Within_Cluster"] = [ "cluster "+ str(cl) for cl in clusters_d ]
itemizedAligned.Within_Cluster.value_counts()

cluster 1    398
cluster 2     25
cluster 3     12
cluster 4      1
Name: Within_Cluster, dtype: int64

In [41]:
itemizedAligned

,Accession,Geo_Location,Collection_Date,Length,Sequence,Alignment,Within_Cluster
0,NC_045512,China,2019-12,29903,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...,cluster 1
1,MT308704,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...,cluster 1
2,MT308703,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...,cluster 1
3,MT308702,USA: NC,2020-04,29834,AACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAAC...,-----------------------------AACAAACCAACCAACTT...,cluster 1
4,MT304479,USA: GA,3/3/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...,cluster 1
...,...,...,...,...,...,...,...
431,MN988713,USA: Illinois,1/21/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...,cluster 1
432,MN938384,China: Shenzhen,1/10/2020,29838,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...,------------------------------------CAACCAACTT...,cluster 1
433,MN975262,China,1/11/2020,29891,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...,cluster 1
434,MN985325,USA,1/19/2020,29882,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,----ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTT...,cluster 1


In [ ]:
#Run Cell to Initialize ETE 3 Tree Visualization 
#Zoom In & Should assume all branch lengths are 1.0 for readability
t = Tree("aprtree.phy")
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180
t.show(tree_style=ts)